Start with a toy dataset used in one of the first diffusion papers [Sohl-Dickstein et.al. 2015], where $$ \Kset \subset \R^2 $$ are points sampled from a spiral. We first construct and visualize this dataset

In [ ]:
dataset = Swissroll(np.pi/2, 5*np.pi, 100)
loader  = DataLoader(dataset, batch_size=2048)